In [1]:
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
import pyro
import pyro.distributions as dist
from tqdm import tqdm

# Load the JSON file
file_path = "/home/pranav24/cs-546-project/SSR/Latest_Weights/QA_Weights/task024_cosmosqa_answer_generation.json"
with open(file_path, "r") as f:
    data = json.load(f)

# Extract input-output pairs from JSON
instances = data["Instances"][2500:5000]
inputs = [instance["input"] for instance in instances]
outputs = [instance["output"][0] for instance in instances]

# Split the data into train and test sets
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(
    inputs, outputs, test_size=0.2, random_state=42
)

# Convert data to Hugging Face Dataset format
train_ds = Dataset.from_dict({"input": train_inputs, "output": train_outputs})
test_ds = Dataset.from_dict({"input": test_inputs, "output": test_outputs})

# Tokenizer setup
base_model_path = "meta-llama/Meta-Llama-3-8B"  # Replace with actual model path
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Check if tokenizer has a padding token, if not, set the eos_token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    labels = tokenizer(
        examples["output"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

# Tokenize datasets
tokenized_train_ds = train_ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_test_ds = test_ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])

# Convert datasets to PyTorch format
tokenized_train_ds.set_format("torch")
tokenized_test_ds.set_format("torch")

# Create DataLoaders
batch_size = 16  # Adjust as needed
train_loader = DataLoader(tokenized_train_ds, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(tokenized_test_ds, batch_size=batch_size)

# Define the model and load weights
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)
fine_tuned_weights_path = "/home/pranav24/cs-546-project/finetuned-weights-LoRA-EVCL-Final-Task2_EVCL_best"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, fine_tuned_weights_path)
pyro.get_param_store().load('pyro_param_store_task2_vcl_best.pt')

# Ensure compatibility with the unchanged part of the code
DEVICE = model.device

# Generate predictions
predictions = []
references = []
sampled_weights_log = []  # Store sampled weights

print("Generating predictions:")

for i in tqdm(range(0, len(test_inputs), batch_size)):  # Loop in batches
    batch_inputs = test_inputs[i:i + batch_size]
    batch_references = test_outputs[i:i + batch_size]

    # Tokenize the inputs in a batch
    inputs_tokenized = tokenizer(batch_inputs, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            # Apply Pyro parameters to LoRA layers
            for name, module in model.named_modules():
                if hasattr(module, "lora_A"):
                    for key in module.lora_A:
                        loc = pyro.param(f"{name}.lora_A.{key}_loc")
                        scale = pyro.param(f"{name}.lora_A.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_A.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_A[key].weight.data.copy_(sampled_weight)

                if hasattr(module, "lora_B"):
                    for key in module.lora_B:
                        loc = pyro.param(f"{name}.lora_B.{key}_loc")
                        scale = pyro.param(f"{name}.lora_B.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_B.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_B[key].weight.data.copy_(sampled_weight)

            # Generate predictions using the tokenized inputs
            generated_ids = model.generate(
                input_ids=inputs_tokenized["input_ids"],
                attention_mask=inputs_tokenized["attention_mask"],
                max_length=1000,  # Adjust as needed
                num_return_sequences=1,
                do_sample=True  # Optional: Sampling for diverse generations
            )

        # Decode generated IDs
        batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend(batch_predictions)
        references.extend(batch_references)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/pranav24/cs-546-project/venv/lib/python3.11/site-packages/pyro/params/param_store.py:334: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(input_file, 

Generating predictions:


  0%|          | 0/32 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_9238/2559409691.py:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  3%|▎         | 1/32 [02:18<1:11:26, 138.28s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  6%|▋         | 2/32 [04:37<1:09:28, 138.96s/it]Setting `pad_token_id` to `eos_token_id

In [35]:
# test_inputs=["Context: Then I drove all around to the used DVD stores to get season 2. ( which I found for 20 bucks! ) I have been watching a disk a night, and not going to bed until 3 am because I knew that the new season was starting soon and I didn't want to be behind. I just finished season 2 and was on the edge of my seat the whole last episode. Yesterday I taped what was on tv. Question: What does the narrator think about the price of the DVD set purchased?"]
test_inputs=["Context: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played . Question: In the future , will this person go to see other bands play ?"]
batch_size=1
final_answer=[]

base_model_path = "meta-llama/Meta-Llama-3-8B"
# fine_tuned_weights_path = "/home/pranav24/cs-546-project/finetuned-weights-LoRA-EVCL-Final-Task1_VCL_best"

fine_tuned_weights_path="/home/pranav24/cs-546-project/SSR/Latest_Weights/QA_Weights/finetuned-weights/QA_Final"
pyro.clear_param_store()

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, fine_tuned_weights_path)
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')

# Ensure compatibility with the unchanged part of the code
DEVICE = model.device


for i in tqdm(range(0, len(test_inputs), batch_size)):  # Loop in batches
    batch_inputs = test_inputs[i:i + batch_size]
    # batch_references = test_outputs[i:i + batch_size]

    # Tokenize the inputs in a batch
    inputs_tokenized = tokenizer(batch_inputs, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            # Apply Pyro parameters to LoRA layers
            # for name, module in model.named_modules():
            #     if hasattr(module, "lora_A"):
            #         for key in module.lora_A:
            #             loc = pyro.param(f"{name}.lora_A.{key}_loc")
            #             scale = pyro.param(f"{name}.lora_A.{key}_scale")
            #             sampled_weight = pyro.sample(
            #                 f"{name}.lora_A.{key}",
            #                 dist.Normal(loc, scale).to_event(loc.dim())
            #             )
            #             sampled_weights_log.append(
            #                 (name, key, sampled_weight.clone().cpu().numpy())
            #             )
            #             module.lora_A[key].weight.data.copy_(sampled_weight)

            #     if hasattr(module, "lora_B"):
            #         for key in module.lora_B:
            #             loc = pyro.param(f"{name}.lora_B.{key}_loc")
            #             scale = pyro.param(f"{name}.lora_B.{key}_scale")
            #             sampled_weight = pyro.sample(
            #                 f"{name}.lora_B.{key}",
            #                 dist.Normal(loc, scale).to_event(loc.dim())
            #             )
            #             sampled_weights_log.append(
            #                 (name, key, sampled_weight.clone().cpu().numpy())
            #             )
            #             module.lora_B[key].weight.data.copy_(sampled_weight)

            # Generate predictions using the tokenized inputs
            generated_ids = model.generate(
                input_ids=inputs_tokenized["input_ids"],
                attention_mask=inputs_tokenized["attention_mask"],
                max_length=512,  # Adjust as needed
                num_return_sequences=1,
                do_sample=True  # Optional: Sampling for diverse generations
            )

        # Decode generated IDs
        final_batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        final_answer.extend(final_batch_predictions)
        # predictions.extend(batch_predictions)
        # references.extend(batch_references)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_9238/197527174.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 1/1 [01:55<00:00, 115.33s/it]


In [29]:
inputs_tokenized["input_ids"]

tensor([[128000,   2014,     25,   7839,  10846,   5111,    323,   1732,    445,
            551,    358,   5602,   2225,    315,   1521,  21562,   8079,   3814,
           1174,    323,    814,   2225,  42423,    757,   3201,    662,  14243,
            662,   7839,  10846,   5111,    374,  45166,    323,   3727,    757,
          15648,    662,    358,   2216,    649,    539,   1520,    719,    387,
           6380,    994,    358,   9020,    311,   1124,   2652,    358,   1781,
            433,    364,     82,    279,   2144,    430,    814,   9508,    779,
           6380,   5694,    994,    814,   6476,    662,  16225,     25,    763,
            279,   3938,   1174,    690,    420,   1732,    733,    311,   1518,
           1023,  21562,   1514,    949]], device='cuda:0')

In [16]:
# batch_inputs

["Context: Then I drove all around to the used DVD stores to get season 2. ( which I found for 20 bucks! ) I have been watching a disk a night, and not going to bed until 3 am because I knew that the new season was starting soon and I didn't want to be behind. I just finished season 2 and was on the edge of my seat the whole last episode. Yesterday I taped what was on tv. Question: What does the narrator think about the price of the DVD set purchased?"]

In [36]:
final_answer

["Context: Good Old War and person L : I saw both of these bands Wednesday night, and they both blew me away. seriously. Good Old War is acoustic and makes me smile. I really can not help but be happy when I listen to them ; I think it's the fact that they seemed so happy themselves when they played. Question: In the future, will this person go to see other bands play?. \nQuestion: What type of person is this?The person is a fan of music. They go to see bands play. They like Good Old War and person L. They are a happy person. They enjoy music. They like to listen to bands play. They enjoy listening to music. They like to see bands perform. They are a happy person. They enjoy listening to music. They enjoy seeing bands perform. They are a fan of music. They like to see bands play. They like Good Old War and person L. They are a happy person. They like to see bands perform. They like to listen to music. They are a fan of music. They like to see bands play. They like Good Old War and pers

In [13]:
import json

# Create a dictionary to store the data
data = {
    "batch_predictions": batch_predictions,
    "predictions": predictions,
    "references": references
}

# Save to a JSON file
with open("predictions_EVCL_Task2.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("Data saved to predictions.json")



Data saved to predictions.json


In [23]:
file_path = "/home/pranav24/cs-546-project/SSR/Latest_Weights/QA_Weights/task024_cosmosqa_answer_generation.json"
with open(file_path, "r") as f:
    data = json.load(f)

# Extract input-output pairs from JSON
instances = data["Instances"][2500:5000]
inputs = [instance["input"] for instance in instances]
outputs = [instance["output"][0] for instance in instances]

# Split the data into train and test sets
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(
    inputs, outputs, test_size=0.2, random_state=42
)

In [25]:
print(test_inputs[0])

Context: When Derek joined her , the door man unhooked the rope to let them through , still watching Lucy with undisguised lust as they walked in . A look of utter confusion shown on Derek 's features as they were granted entrance into the club . He draped an arm over Lucy 's shoulders and they made their way through the crowds of people . " So how 'd you do that ?. 
Question: What may be the reason for Derek 's confusion ?
